In [6]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from mechanistic_model import ODE_model, prepare_ODE_model_data, fit_ODE_model
from scipy.integrate import odeint
from pygom import SimulateOde
import random
import statsmodels.api as sm
import statsmodels.formula.api as smf
# setting up the plotting
plt.rcParams['figure.figsize'] = [20, 10]
plt.rcParams['font.size'] = 12 

# Generating ABC posteriors

The model used to simulated epidemics is stored in mechanisitic_model.py alongside other utility functions. Although the model is relatively simple, care has been tkaen to ensure erlang distributed exposed and infected compartments, as well as accounting for detcted and undetected cases.

All dates are inputted relative to the first day in the dataset. The 'assumed_lag' parameter dictates how many post-intervention data points are included in the fitting process. This allows for the incorporation of more data points, but care should betaken to ensure it remains below the incubaiton period of the disease.

In [11]:
# Funcitons for weighting and parameterising the NegBinomLoss function

def calculate_fit_weights(data):
    weights = [1]
    
    for x in range(1, len(data)):
        if data[x] == 0:
            weight = 0.0001
        else:
            weight = (data[x] - data[x-1])/data[x]
        
        if weight == 0:
            weight = 0.00001 # Note weights can not be zero
        weights.append(weight)
    return weights

# Estimating alpha for NegBinomLoss 
# adapted from https://towardsdatascience.com/negative-binomial-regression-f99031bb25b4
def fit_poisson(endog, exog, weights):
    poiss = sm.GLM(endog, exog, family=sm.families.Poisson(), weights=weights).fit()
    lambdas = poiss.mu
    df_train = pd.DataFrame(columns=['BB_count', 'BB_lambda', 'Aux_OLS_dep'])
    df_train['BB_count']=endog
    df_train['BB_lambda']=lambdas
    return df_train


def fit_auxiliary_OLS(df_train):
    df_train['Aux_OLS_dep'] = df_train.apply(lambda x:\
                                             ((x['BB_count']\
                                               - x['BB_lambda'])**2 \
                                               - x['BB_count'])\
                                               / x['BB_lambda'], axis=1)   
    ols_expr = """Aux_OLS_dep ~ BB_lambda - 1"""
    aux_olsr_results = smf.ols(ols_expr, df_train).fit()
    return abs(aux_olsr_results.params.BB_lambda)


def estimate_over_dispersion(data, weights):
    endog = data
    exog = np.linspace(0, len(data)-1, len(data))
    df_train = fit_poisson(endog, exog, weights)
    alpha = fit_auxiliary_OLS(df_train)
    k = 1/alpha
    return k

In [12]:
nat_pop_dict = {'DK':5.8e6, 'NO':5.37e6, 'SE':1.02e7}
int_day_dict = {'SE':10, 'DK':11, 'NO':6}

In [13]:
# Specify parameters
country = 'DK' # Two letter country code
metric = 'new_hosps' # 'new_cases' or 'new_hosps'
N0 = nat_pop_dict[country]
start_date = 0 # Pretty much always zero...
int_date = int_day_dict[country]-start_date # Date of closures
end_date = 28 # Final date of simulation
assumed_lag = 5 # 5 days for cases and hospitalisations
num_sims = 150 # Number of posterior parameterisations

In [16]:
# Load in the data and extract the useful parts
nat_data = pd.read_csv('raw_data/{}_data.csv'.format(country))
nat_data = nat_data.filter([metric]) 
nat_data = nat_data.dropna(axis=0)
# Want to save this as a seperate dataframe
df = pd.DataFrame(columns=[metric])#
df[metric] = [int(x) for x in nat_data[metric]]
df.to_csv('GP_input_data/{}_{}.csv'.format(country, metric))

## ABC fitting
Functions initialising and running the ABC fitting process can be found in mechanisitic_model.py. Posteriors are estimated for all parameters, with the exception of $\alpha_i$, which is set to 1.7, such that $\sum_{i=0}^2\alpha_i = 5.1$ - a reasonable approximation of the known incubation period of Covid-19. Initial Exposed populaiton is also estimated during the ABC fitting process. 

In [17]:
# Preparations for ABC fitting
fit_data, fit_end_point = prepare_ODE_model_data(df=nat_data, 
                                                 fit_to=metric, 
                                                 N_to_fit=int_date+assumed_lag)
# Estimate over dispersion (k) of fitting data and calculate weightings
weights = calculate_fit_weights(fit_data)
k = estimate_over_dispersion(fit_data, weights)

In [18]:
# Run ABC fitting. 10 generations is usually sufficient. 
ode_model, fitted_model = fit_ODE_model(full_data=nat_data,
                                         fit_data=fit_data, 
                                         fit_to=metric, 
                                         fit_end_point=fit_end_point,
                                         weights=None,
                                         k=k*np.ones(len(fit_data)),
                                         N0=N0,
                                         generations=10)        

<class 'numpy.int64'>
BOX BOUNDS: [(0, 3), (0, 5), (0, 1), (0, 21), (0, 21), (0.25, 4), (0.25, 4), (0.25, 4), (0.25, 4), (0.25, 4), (0.25, 1)]
Using NegBinom loss


/home/ubuntu/anaconda3/envs/InterventionsNNpy37/lib/python3.7/site-packages/pygom-0.1.8.dev15+g8c6d4ce.d20201111-py3.7-linux-x86_64.egg/pygom/utilR/distn.py:515: RuntimeWarning: divide by zero encountered in log
  logpmf_p4= x*(np.log(mu) - np.log(k + mu))
/home/ubuntu/anaconda3/envs/InterventionsNNpy37/lib/python3.7/site-packages/pygom-0.1.8.dev15+g8c6d4ce.d20201111-py3.7-linux-x86_64.egg/pygom/utilR/distn.py:515: RuntimeWarning: invalid value encountered in multiply
  logpmf_p4= x*(np.log(mu) - np.log(k + mu))


Generation 1 
 tolerance = inf 
 acceptance rate = 100.00%

Generation 2 
 tolerance = 125.28987 
 acceptance rate = 2.96%

Generation 3 
 tolerance = 59.34215 
 acceptance rate = 3.31%

Generation 4 
 tolerance = 43.91563 
 acceptance rate = 2.56%

Generation 5 
 tolerance = 39.23275 
 acceptance rate = 2.18%

Generation 6 
 tolerance = 37.30372 
 acceptance rate = 2.11%

Generation 7 
 tolerance = 36.43993 
 acceptance rate = 1.61%

Generation 8 
 tolerance = 35.93679 
 acceptance rate = 1.31%

Generation 9 
 tolerance = 35.57646 
 acceptance rate = 1.06%

Generation 10 
 tolerance = 35.32308 
 acceptance rate = 1.11%



# Simulation 

A random selection of posteriors generated by the ABC fitting method are used to produce a simulated epidemic curve. Since the ABC fitting is done up to (or just past) intervention, it can be assumed that these can act as counter factuals to the observed cases under intervention. 


In [ ]:
model = ode_model
times = np.linspace(0,end_date-1,end_date)

curve_idx_dict = {'cases':7, 'hospitalisations':7, 'tot_hosps':5, 'new_cases':5}
curve_idx = curve_idx_dict[metric]
n = 0
pred = np.zeros((num_sims,len(times)))
sim = np.zeros((num_sims,len(times)))
I0 = 0
Eb0 = 0
Ec0 = 0
Id0 = 0
Iu0 = 0
R0  = 0
Id_cum0 = 0
null_series = 0

for i in range(num_sims):
    print('Simulating posterioir {}'.format(i))
    p_beta, p_gamma, p_kappa, p_delta1, p_delta2, p_Ea0, p_Eb0, p_Ec0, p_I0, p_Id0, p_Iu0 = fitted_model.res[i]
    model.parameters = {'beta':p_beta,
                        'alpha0':1.6,
                        'alpha1':1.6,
                        'alpha2':1.6,
                        'gamma':p_gamma, 
                        'kappa':p_kappa,
                        'delta1':p_delta1,
                        'delta2':p_delta2,
                        'N0':N0}

    solution = odeint(model.ode,[N0-(p_Ea0+p_Eb0+p_Ec0+p_I0+p_Id0+p_Iu0+R0), 
                                 np.round(p_Ea0),np.round(p_Eb0),np.round(p_Ec0), np.round(p_I0), np.round(p_Id0)
                                 , np.round(p_Iu0), Id_cum0, R0],times).T[curve_idx]
    if sum(solution) != 0:
        pred[i] = solution


In [ ]:
fit_data

In [ ]:
# Store the simulated data
simulated_data = {}
for idx, data in enumerate(pred):
    simulated_data.update({idx:data})

In [ ]:
# Save the results
det_df = pd.DataFrame.from_dict(simulated_data)
# Generally only interested in stochastic data, although deterministic can be useful for longer sims.
det_df.to_csv('ABC_posteriors/posteriors_{}_{}.csv'.format(metric, country))

In [ ]:
# Plot to check everything looks sensible.
#start_date = int_date-10
df = pd.read_csv('ABC_posteriors/posteriors_{}_{}.csv'.format(metric, country))
for x in range(100):
    plt.plot(pred[x][start_date:], 'r', alpha=0.2)
plt.plot(times, nat_data[metric][:end_date], 'x', label = 'Observed data')
plt.plot(np.median(pred, axis=0)[start_date:end_date], 'k--', label = 'Median forecast')
plt.title('ABC Posteriors for {}'.format(country))
plt.ylabel('Cumulative cases')
plt.xlabel('Days')
plt.axvline(x=int_date, linewidth=4, color="r")
#plt.axvline(x=int_date+assumed_lag, linewidth=2, color="r")
plt.legend()
#plt.ylim([0, 10000])
#plt.xlim([0,30])
plt.yscale('log')

In [ ]:
## Check that num observations and num simulated data points are the same!
assert len(pred[0])==len(nat_data[metric][start_date:end_date])

In [ ]:
# Do some cleaning up of the dataframe to get rid of unnecessary bits
fit_data = [int(x) for x in nat_data[metric]]
fit_dict = {metric:fit_data}
fit_df = pd.DataFrame.from_dict(fit_dict)
fit_df.to_csv('GP_input_data/{}_{}.csv'.format(country, metric, assumed_lag))